In [1]:
import requests
import pandas as pd 
import json
import pymongo
from pymongo import MongoClient

In [15]:
table_name = 'Employment'
series_type = '05'

In [3]:
df_local_msa = pd.read_excel('../Local_Area/local_area_msa.xlsx')
df_local_msa

,CBSA,MSA_Name,area_code
0,11500,"Anniston-Oxford, AL",MT0111500000000
1,12220,"Auburn-Opelika, AL",MT0112220000000
2,13820,"Birmingham-Hoover, AL",MT0113820000000
3,19300,"Daphne-Fairhope-Foley, AL",MT0119300000000
4,19460,"Decatur, AL",MT0119460000000
...,...,...,...
379,39100,"Poughkeepsie-Newburgh-Middletown, NY",DV3620524000000
380,39150,"Prescott Valley-Prescott, AZ",MT0439140000000
381,39300,"Providence-Warwick, RI-MA",MT4477200000000
382,44140,"Springfield, MA",MT2578100000000


In [4]:
# set up dataframe and create a list of the series for this dataset
df = pd.DataFrame(columns=["series id","year","period","period_name","value"])
series = []
for index, row in df_local_msa.iterrows():
    # construct series id by concatenating: Prefix + MSA Area Code + Measure Code
    series.append('LAU' + row['area_code'] + series_type)

In [5]:
series[8]

'LAUMT012662000000005'

In [8]:
# function to request data from the BLS api
def request_series(series_list):
    #  print(series_list)
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": series_list,"startyear":"2010", "endyear":"2019","registrationkey":"<key here>"})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    if json_data['status'] == 'REQUEST_SUCCEEDED':
        add_to_df(json_data)
    else:
        print(json_data['status'])

In [7]:
# function to add data to the dataframe
def add_to_df(data):
    for series in data['Results']['series']:
        seriesId = series['seriesID']
        for item in series['data']:
            year = item['year']
            period = item['period']
            period_name = item["periodName"]
            value = item['value']
            if 'M01' <= period <= 'M12':
                df.loc[len(df)] = [seriesId,year,period,period_name,value]

In [9]:
# Break up the series codes into chunks of 50, request it, get it back, and load it in a dataframe
lister = [series[i:i + 50] for i in range(0, len(series), 50)]

for i in range(0, len(lister)):
    request_series(lister[i])
    print(i)  # prints the number of the 'set of 50'

0
1
2
3
4
5
6
7


In [10]:
df

,series id,year,period,period_name,value
0,LAUMT011150000000005,2019,M12,December,44930
1,LAUMT011150000000005,2019,M11,November,45076
2,LAUMT011150000000005,2019,M10,October,45127
3,LAUMT011150000000005,2019,M09,September,44455
4,LAUMT011150000000005,2019,M08,August,44535
...,...,...,...,...,...
46075,LAUMT257960000000005,2010,M05,May,309891
46076,LAUMT257960000000005,2010,M04,April,309823
46077,LAUMT257960000000005,2010,M03,March,307514
46078,LAUMT257960000000005,2010,M02,February,306438


In [11]:
df.groupby('series id').count().sort_values(by='value')

,year,period,period_name,value
series id,,,,
LAUDV362052400000005,120,120,120,120
LAUMT394966000000005,120,120,120,120
LAUMT394826000000005,120,120,120,120
LAUMT394578000000005,120,120,120,120
LAUMT394422000000005,120,120,120,120
...,...,...,...,...
LAUMT181402000000005,120,120,120,120
LAUMT174410000000005,120,120,120,120
LAUMT174042000000005,120,120,120,120


In [12]:
df_cbsa = df_local_msa[['CBSA', 'area_code']].copy()
df_cbsa['series id'] = 'LAU' + df_cbsa['area_code'] + series_type
df_cbsa.head()

,CBSA,area_code,series id
0,11500,MT0111500000000,LAUMT011150000000005
1,12220,MT0112220000000,LAUMT011222000000005
2,13820,MT0113820000000,LAUMT011382000000005
3,19300,MT0119300000000,LAUMT011930000000005
4,19460,MT0119460000000,LAUMT011946000000005


In [13]:
df_combined = pd.merge(df_cbsa, df, on="series id")
df_combined.drop(columns=['area_code'], inplace=True)
df_combined

,CBSA,series id,year,period,period_name,value
0,11500,LAUMT011150000000005,2019,M12,December,44930
1,11500,LAUMT011150000000005,2019,M11,November,45076
2,11500,LAUMT011150000000005,2019,M10,October,45127
3,11500,LAUMT011150000000005,2019,M09,September,44455
4,11500,LAUMT011150000000005,2019,M08,August,44535
...,...,...,...,...,...,...
46075,49340,LAUMT257960000000005,2010,M05,May,309891
46076,49340,LAUMT257960000000005,2010,M04,April,309823
46077,49340,LAUMT257960000000005,2010,M03,March,307514
46078,49340,LAUMT257960000000005,2010,M02,February,306438


In [16]:
df_combined.to_csv('../CSVs/' + table_name + '.csv', index=False)

In [19]:
# Create instance of MongoClient
client = MongoClient()
# Connection URI
client = MongoClient('<conn string>')
# Select database
db = client['MSA']
# create new collection
collection = db.Employment_raw
# turn dataframe into readable format for mongo
df_dict = df_combined.to_dict(orient='records')
# write dataframe to unemployment_predicted_2024 collection
collection.insert_many(df_dict)